# Example: MNIST Data Set -  Auto Encoder

This is an extension of the [Notebook 1 Tutotial](https://github.com/clonker/bms-summerschool19-tf/blob/master/bms-tensorflow-tutorial-1.ipynb).

[Autoencoder (wiki)](https://en.wikipedia.org/wiki/Autoencoder)

## Prepare Notebook

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

%matplotlib inline

## Load + Prepare Data

Let us prepare the data as in the [previous](https://github.com/juanitorduz/math_deep_learning_summer_school19/blob/master/keras/mnist_sequential.ipynb) notebook.

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = np.array(x_train, dtype=np.float32)
x_test = np.array(x_test, dtype=np.float32)

Let us see the shape of the training data:

In [5]:
x_train.shape

(60000, 28, 28)

In [22]:
batch_size = 32

train_ds = tf.data.Dataset.from_tensor_slices((x_train, x_train)).shuffle(1024).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, x_test)).shuffle(1024).batch(batch_size)

## Class Definition

In [11]:
class Encoder(tf.keras.Model):
    def __init__(self, latent_dim):
        super(Encoder, self).__init__()
        self.dense = tf.keras.layers.Dense(
            units=latent_dim, activation=tf.nn.relu, input_shape=(None, 28, 28,))
    
    def call(self, inputs):
        flat = tf.reshape(inputs, shape=[-1, 28*28])
        latent = self.dense(flat)
        return latent

In [12]:
class Decoder(tf.keras.Model):
    def __init__(self, latent_dim):
        super(Decoder, self).__init__()
        self.dense = tf.keras.layers.Dense(
            units=28*28, activation=tf.nn.relu, input_shape=(None, latent_dim)) 
    
    def call(self, inputs):
        flat = self.dense(inputs)
        image = tf.reshape(flat, [-1, 28, 28])
        return image

In [13]:
class Autoencoder(tf.keras.Model):
    def __init__(self, latent_dim):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder(latent_dim)
        self.decoder = Decoder(latent_dim)

    def call(self, inputs):
        latent = self.encoder(inputs)
        reconstructed = self.decoder(latent)
        return reconstructed

In [23]:
dim = 7
ae = Autoencoder(dim)
ae.build(input_shape=(batch_size, 28, 28))

ae.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.MeanSquaredError()]
)

ae.summary()

Model: "autoencoder_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_2 (Encoder)          multiple                  5495      
_________________________________________________________________
decoder_2 (Decoder)          multiple                  6272      
Total params: 11,767
Trainable params: 11,767
Non-trainable params: 0
_________________________________________________________________


In [19]:
model_history = ae.fit(
    train_ds, 
    epochs=5, 
    validation_data=test_ds
)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0471 - mean_squared_error: 0.0546 - val_loss: 0.0438 - val_mean_squared_error: 0.0438
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0435 - mean_squared_error: 0.0435 - val_loss: 0.0435 - val_mean_squared_error: 0.0435
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0434 - mean_squared_error: 0.0433 - val_loss: 0.0434 - val_mean_squared_error: 0.0435
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0434 - mean_squared_error: 0.0433 - val_loss: 0.0434 - val_mean_squared_error: 0.0434
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0434 - mean_squared_error: 0.0433 - val_loss: 0.0434 - val_mean_squared_error: 0.0434
